<div style="margin: 0 auto; padding: 20px;">

# ROI Selection

In this initial Jupyter workflow within PyCCAPT, we will guide you through the process of cropping atom probe data, whether it's originally collected using PyCCAPT or in various other formats such as EPOS, POS, ATO, and CSV. This workflow is designed to help you efficiently manage your atom probe data, focusing on both temporal and spatial cropping techniques. Additionally, we will cover essential calculations, including raw MC (Mass-to-Charge Ratio), pulses per ion, and ions per pulse. Lastly, you can explore how to save the cropped data in a range of formats, including PyCCAPT's native HDF5 format, EPOS, POS, ATO, and CSV, to suit your specific needs and preferences.
</div>

In [2]:
# Activate intractive functionality of matplotlib
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
from ipywidgets import fixed
from ipywidgets import interact_manual
from ipywidgets import widgets

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools, tof_tools

<div style="margin: 0 auto; padding: 20px;">
By clicking on the button below, you can select the dataset file you want to crop. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [3]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px;">
In case of recieving the error about pytable library, please install the pytables library by running the cell below. Befor running the cell, please uncomment the cell by removing the # sign at the begining of the cell.
</div>


In [4]:
# check pytables library and install it
# try:
#     import tables
# except:
#     !conda install --yes --prefix {sys.prefix} pytables

<div style="margin: 0 auto; padding: 20px;">
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [5]:
# create object for selection of instrument specifications of the dataset
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flightPathLength_d, t0_d, max_mc)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=54.0, description='t0:')

FloatText(value=400.0, description='Max mc:')

In [24]:
# Calculate the maximum possible time of flight (TOF)
max_tof = int(tof_tools.mc2tof(max_mc.value, 1000, 0, 0, flightPathLength_d.value))
print('The maximum possible TOF is:', max_tof, 'ns')
print('=============================')
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode.value
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/' + variables.dataset_name + '/load_crop/'
variables.result_data_name = variables.dataset_name 
variables.result_path = dataset_main_path + '/' + variables.dataset_name + '/load_crop/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)

print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')

# Create data farame out of hdf5 file dataset
dld_group_storage = data_tools.load_data(dataset_path, tdc.value, mode='raw')

# Remove the data with tof greater thatn Max TOF or below 0 ns
data = data_tools.remove_invalid_data(dld_group_storage, max_tof)
print('Total number of Ions:', len(data))

The maximum possible TOF is: 11069 ns
The data will be saved on the path: C:/Users/mehrp/PycharmProjects/pyccapt/tests/data/data_130_Sep-19-2023_14-58_W_12fs/load_crop/
The dataset name after saving is: data_130_Sep-19-2023_14-58_W_12fs
The figures will be saved on the path: C:/Users/mehrp/PycharmProjects/pyccapt/tests/data/data_130_Sep-19-2023_14-58_W_12fs/load_crop/
{'apt': ['high_voltage', 'num_events', 'time_counter'], 'dld': ['high_voltage', 'pulse', 'start_counter', 't', 'x', 'y'], 'tdc': ['ch0', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7'], 'time': ['time_h', 'time_m', 'time_s']}
The number of data over max_tof: 0
Total number of Ions: 1431644


In [25]:
data

,high_voltage (V),pulse,start_counter,t (ns),x_det (cm),y_det (cm)
0,0.00000,2.072519e+12,0,4988.0,1.10450,3.011370
1,0.00000,2.072519e+12,0,2462.0,-0.29357,-0.582315
2,0.00000,2.072519e+12,0,4294.0,1.08844,-1.349060
3,0.00000,2.072519e+12,0,0.0,0.00000,0.000000
4,0.00000,2.072519e+12,0,0.0,0.00000,0.000000
...,...,...,...,...,...,...
1431639,9000.49707,1.524632e+13,0,5181.0,1.10905,3.081950
1431640,9000.49707,1.524632e+13,0,0.0,0.00000,0.000000
1431641,9000.49707,1.524632e+13,0,4423.0,2.52355,2.734570
1431642,9000.49707,1.524632e+13,0,724.0,1.91241,-0.255721


<div style="margin: 0 auto; padding: 20px;">

# Temporal crop

Select the data by drawing a rectangle over the experiment history. Experiment history is a 2D histogram of the time of flight of the ions versus sequence of evaporation. The experiment history is plotted by clicking on the button below te cell.
</div>


In [26]:
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(False), data_crop=fixed(True), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='hist_ini'));


C:\Users\mehrp\AppData\Local\Temp\ipykernel_6112\4188335034.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=11069.0, description='max_tof'), FloatText(value=1.0, description='frac'…

<div style="margin: 0 auto; padding: 20px;">

Boundaries of the selected(cropped) part of the graph is printed below
</div>

In [27]:
# Plot and selected experiment history
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(True), data_crop=fixed(False), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='hist_rect'));


C:\Users\mehrp\AppData\Local\Temp\ipykernel_6112\2323789099.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=11069.0, description='max_tof'), FloatText(value=1.0, description='frac'…

In [28]:
# Crop the dataset
print('Min Idx:', variables.selected_x1, 'Max Idx:', variables.selected_x2)
data_crop_t = data_loadcrop.crop_dataset(data, variables)

Min Idx: 13044.076819093898 Max Idx: 1373871.2377287196


<div style="margin: 0 auto; padding: 20px;">

# Spacial crop

Select the region of maximum concentration of Ions in the below plotted graph to utilize relevant data. To crop you can draw a circle over the filed desorption map. The field desorption map is a 2D histogram of the time of flight of the ions versus the position of the ions on the detector. The field desorption map is plotted by clicking on the button below the cell.
</div>

In [29]:
# Plot and select the FDM
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
               draw_circle=fixed(False), data_crop=fixed(True), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm_ini'));

C:\Users\mehrp\AppData\Local\Temp\ipykernel_6112\1567814332.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

<div style="margin: 0 auto; padding: 20px;">

The region selected in the previous step is displayed below.
</div>

In [30]:
# plot selected area in FDM
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
                draw_circle=fixed(True), data_crop=fixed(False), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm_circle'));

C:\Users\mehrp\AppData\Local\Temp\ipykernel_6112\3314188428.py:2: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

In [31]:
# Crop the dataset
print('center x:', variables.selected_x_fdm, 'center y:', variables.selected_y_fdm)
print('Radios:', variables.roi_fdm)
if variables.roi_fdm > 0:
    data_crop_spatial = data_loadcrop.crop_data_after_selection(data_crop_t, variables)
else:
    print('select the data spacialy from cell below')

center x: -0.0469302144613164 center y: 0.055442633998466384
Radios: 3.739585865675548


c:\users\mehrp\pycharmprojects\pyccapt\pyccapt\calibration\data_tools\data_loadcrop.py:327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop.drop(np.where(mask_fdm)[0], inplace=True)


<div style="margin: 0 auto; padding: 20px;">

Calculate pulses since the last event pulse and ions per pulse.
</div>

In [33]:
pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(data_crop_spatial)

# add two calculated array to the croped dataset
data_crop_spatial['pulse_pi'] = pulse_pi.astype(np.uintc)
data_crop_spatial['ion_pp'] = ion_pp.astype(np.uintc)


C:\Users\mehrp\AppData\Local\Temp\ipykernel_6112\949659944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop_spatial['pulse_pi'] = pulse_pi.astype(np.uintc)
C:\Users\mehrp\AppData\Local\Temp\ipykernel_6112\949659944.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_crop_spatial['ion_pp'] = ion_pp.astype(np.uintc)


$\text {The percentage of loss in ROI selection process.}$

In [34]:
# save the cropped data
print('tof Crop Loss {:.2f} %'.format((100 - (len(data_crop_spatial) / len(data)) * 100)))

tof Crop Loss 8.92 %


In [35]:
# exctract needed data from Pandas data frame as an numpy array
dld_highVoltage = data_crop_spatial['high_voltage (V)'].to_numpy()
dld_pulse = data_crop_spatial['pulse'].to_numpy()
dld_t = data_crop_spatial['t (ns)'].to_numpy()
dld_x = data_crop_spatial['x_det (cm)'].to_numpy()
dld_y = data_crop_spatial['y_det (cm)'].to_numpy()

<div style="margin: 0 auto; padding: 20px;">

In the next cell by changing the t<sub>0</sub> value you can correct the position of H<sub>1</sub>. this correction would be helpful for the position of the peaks in the m/c calibration process. H<sub>2</sub>O
</div>

In [36]:
def fine_tune_t_0(variables, t0_d, bin_size, mode, prominence, distance, percent, selector, plot, figname, lim, peaks_find_plot):
    variables.mc_calib = mc_tools.tof2mc(dld_t, t0_d, dld_highVoltage, dld_x, dld_y, flightPathLength_d.value, dld_pulse, mode=pulse_mode.value)
    tools.mc_hist_plot(variables, bin_size, mode, prominence, distance, percent, selector, plot, figname, lim, peaks_find_plot)
    
interact_manual(fine_tune_t_0, variables=fixed(variables), t0_d=widgets.FloatText(value=t0_d.value), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc')]), prominence=widgets.IntText(value=10), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=fixed(True));

C:\Users\mehrp\AppData\Local\Temp\ipykernel_6112\1862098691.py:5: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(fine_tune_t_0, variables=fixed(variables), t0_d=widgets.FloatText(value=t0_d.value), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=29.0, description='t0_d'), FloatText(value=0.1, description='bin_size'),…

In [39]:
data_crop_spatial_back = data_crop_spatial.copy()

In [40]:
data_crop_spatial_back.insert(0, 'x (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(1, 'y (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(2,'z (nm)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(3,'mc_c (Da)', np.zeros(len(dld_t)))
data_crop_spatial_back.insert(4, 'mc (Da)', variables.mc_calib)
data_crop_spatial_back.insert(8,'t_c (ns)', np.zeros(len(dld_t)))

<div style="margin: 0 auto; padding: 20px;">

Remove the data with mc biger than max mc
</div>

In [41]:
# Remove the data with mc biger than max mc
mask = (data_crop_spatial_back['mc (Da)'].to_numpy() > max_mc.value)
print('The number of data over max_mc:', len(mask[mask==True]))
data_crop_spatial_back.drop(np.where(mask)[0], inplace=True)
data_crop_spatial_back.reset_index(inplace=True, drop=True)

# Remove the data with x,y,t = 0
mask1 = (data_crop_spatial_back['x (nm)'].to_numpy() == 0)
mask2 = (data_crop_spatial_back['y (nm)'].to_numpy() == 0)
mask3 = (data_crop_spatial_back['t (ns)'].to_numpy() == 0)
mask = np.logical_and(mask1, mask2)
mask = np.logical_and(mask, mask3)
print('The number of data with having t, x, and y equal to zero is:', len(mask[mask==True]))
data_crop_spatial_back.drop(np.where(mask)[0], inplace=True)
data_crop_spatial_back.reset_index(inplace=True, drop=True)

The number of data over max_mc: 48054
The number of data with having t, x, and y equal to zero is: 489888


<div style="margin: 0 auto; padding: 20px;">

The final cropped dataset is displayed below.
</div>

In [42]:
data_crop_spatial_back

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,0.0,65.312609,6928.927246,2.072519e+12,0,0.0,1737.0,1.832900,1.659840,0,0
1,0.0,0.0,0.0,0.0,65.529490,6928.927246,2.072519e+12,0,0.0,1749.0,-1.688140,3.110470,0,0
2,0.0,0.0,0.0,0.0,103.605532,6928.927246,2.072519e+12,0,0.0,2180.0,1.036070,2.220390,0,0
3,0.0,0.0,0.0,0.0,0.534732,6928.927246,2.072519e+12,0,0.0,184.0,-0.631777,3.037620,0,0
4,0.0,0.0,0.0,0.0,23.005413,6928.927246,2.072519e+12,0,0.0,1047.0,1.744990,2.855850,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766052,0.0,0.0,0.0,0.0,99.314411,7006.818359,1.498848e+13,0,0.0,2131.0,3.038580,-1.093610,0,0
766053,0.0,0.0,0.0,0.0,99.884976,7006.818359,1.498848e+13,0,0.0,2121.0,0.323457,-1.104690,0,0
766054,0.0,0.0,0.0,0.0,100.409995,7006.818359,1.498848e+13,0,0.0,2148.0,-3.589920,-0.798414,0,0
766055,0.0,0.0,0.0,0.0,100.156349,7006.818359,1.498848e+13,0,0.0,2128.0,-1.045210,-1.607910,0,0


<div style="margin: 0 auto; padding: 20px;">

Te data type of te dataset is displayed below.
</div>

In [43]:
data_crop_spatial_back.dtypes

x (nm)              float64
y (nm)              float64
z (nm)              float64
mc_c (Da)           float64
mc (Da)             float64
high_voltage (V)    float64
pulse               float64
start_counter        uint32
t_c (ns)            float64
t (ns)              float64
x_det (cm)          float64
y_det (cm)          float64
pulse_pi             uint32
ion_pp               uint32
dtype: object

<div style="margin: 0 auto; padding: 20px;">

Save the cropped dataset. You can specify te output format from list below. The output formats are HDF5, EPOS, POS, ATO, and CSV. The output file will be saved in the same directory as the original dataset file in a new directory nammed load_crop.
</div>

In [44]:
interact_manual(data_tools.save_data, data=fixed(data_crop_spatial_back), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

interactive(children=(Dropdown(description='hdf', options=(('True', True), ('False', False)), value=True), Dro…